<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/Acme.jpeg width=100 align="left">

# JWT Auth
---

## Business Value of Capability

### Setup
```vrd1```
---

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root

In [ ]:
vault status

In [ ]:
vault -version

### Demo
---
**Enable the Vault JWT Authentication**

In [ ]:
vault auth enable jwt

**Write some policies**

In [ ]:
vault policy write myproject-staging - <<EOF
# Policy name: myproject-staging
#
# Read-only permission on 'secret/data/myproject/staging/*' path
path "secret/data/myproject/staging/*" {
  capabilities = [ "read" ]
}
EOF

In [ ]:
vault policy write myproject-production - <<EOF
# Policy name: myproject-production
#
# Read-only permission on 'secret/data/myproject/production/*' path
path "secret/data/myproject/production/*" {
  capabilities = [ "read" ]
}
EOF

**Config JWT for a role with bound_claim**

In [ ]:
vault write auth/jwt/role/myproject-staging - <<EOF
{
  "role_type": "jwt",
  "policies": ["myproject-staging"],
  "token_explicit_max_ttl": 60,
  "user_claim": "user_email",
  "bound_claims": {
    "project_id": "9937757",
    "ref": "master",
    "ref_type": "branch"
  }
}
EOF

**NOte:** the project_id `9937757` refers to gitlab.com/bjc project   
**Config JWT for another role with bound_claim**

In [ ]:
 vault write auth/jwt/role/myproject-production - <<EOF
 {
   "role_type": "jwt",
   "policies": ["myproject-production"],
   "token_explicit_max_ttl": 60,
   "user_claim": "user_email",
   "bound_claims_type": "glob",
   "bound_claims": {
     "project_id": "9937757",
     "ref_protected": "true",
     "ref_type": "branch",
     "ref": "auto-deploy-*"
   }
 }
EOF

In [ ]:
vault write auth/jwt/config \
     jwks_url="https://gitlab.com/-/jwks" \
     bound_issuer="gitlab.com"

---
### Populate the KV Engine

In [ ]:
vault kv write secret/myproject/staging/db password=r31nsta11

**Revoke all leases**

In [ ]:
vault kv write secret/myproject/production/db password=r31nsta11

---
## Authenticate to Vault using the JWT

In [ ]:
export VAULT_ADDR=http://54.186.105.168:8200

export VAULT_TOKEN="$(vault write -field=token auth/jwt/login role=myproject-staging jwt=$CI_JOB_JWT)"

export PASSWORD="$(vault kv get -field=password secret/myproject/staging/db)"

echo $PASSWORD

# This will fail because the role myproject-staging can not read secrets from secret/myproject/production/*
export PASSWORD="$(vault kv get -field=password secret/myproject/production/db)"

In [ ]:
export VAULT_ADDR=http://vault.example.com:8200

export VAULT_TOKEN="$(vault write -field=token auth/jwt/login role=myproject-production jwt=$CI_JOB_JWT)"

export PASSWORD="$(vault kv get -field=password secret/myproject/production/db)"
    # Use the secret

echo $PASSWORD

**END**